# Visualizing the saliency maps

Once your classifier is trained you can visualize which pixels where the most relevant to estimate the volume of wine (or any other liquid) in an image. This is what we call the *saliency maps*.

We will use the implementations of different saliency functions from the [deep-viz repository](https://github.com/experiencor/deep-viz-keras) by [experiencor](https://github.com/experiencor) which allows the visualization of saliency maps for keras models.

In [ ]:
import os
import json

import numpy as np
import matplotlib.pylab as plt
from tensorflow.keras.models import load_model

from imgclas import paths, utils, data_utils
from imgclas.data_utils import load_data_splits, k_crop_data_sequence
from imgclas.test_utils import predict
from imgclas.visualization.saliency import GradientSaliency
from imgclas.visualization.guided_backprop import GuidedBackprop
from imgclas.visualization.integrated_gradients import IntegratedGradients
from imgclas.visualization.visual_backprop import VisualBackprop

# User parameters to set
TIMESTAMP = '2024-05-14_074436'                       # timestamp of the model
FOLD = "Fold-2"                                        # Fold of CV to use, set to "" if not use CV
MODEL_NAME = 'final_model.h5'                            # model to use to make the prediction
SPLIT_NAME='test'                                       # Split data used
TOP_K = 2                                               # number of top classes predictions to save

# Set the timestamp
paths.timestamp = TIMESTAMP

# Load training configuration
conf_path = os.path.join(paths.get_conf_dir(), 'conf.json')
with open(conf_path) as f:
    conf = json.load(f)

if FOLD == "":
    filepath = os.path.join(paths.get_checkpoints_dir(), MODEL_NAME)
else:
    filepath = os.path.join(paths.get_checkpoints_dir()+"/"+FOLD, MODEL_NAME)
    
obj=utils.get_custom_objects()

# Load the model
model = load_model(filepath, custom_objects=obj)

Now you can select a file in your computer in which to visualize the saliency maps. 
Possible visualizations include:
* [Vanila gradient](https://arxiv.org/abs/1312.6034)
* [Integrated gradients](https://arxiv.org/abs/1703.01365)

Each of them is accompanied with the corresponding [smoothgrad](https://arxiv.org/abs/1706.03825) version, which improves on any baseline method by adding random noise.

In [ ]:
import json
import pandas as pd

if FOLD == "":
    pred_path = os.path.join(paths.get_predictions_dir(), '{}+{}.json'.format(MODEL_NAME, SPLIT_NAME, TOP_K))
else:
    pred_path = os.path.join(paths.get_predictions_dir()+"/"+FOLD, '{}+{}.json'.format(MODEL_NAME, SPLIT_NAME, TOP_K))
    
with open(pred_path) as f:
    prediccion_data = json.load(f)
    
prediccion_df = pd.DataFrame.from_dict(prediccion_data)
prediccion_df['abs_error'] = abs(prediccion_df.pred_value-prediccion_df.true_value)
prediccion_df = prediccion_df.sort_values(by=['abs_error'])
prediccion_df.columns=['filepath', 'pred_value', 'true_value', 'abs_error']
prediccion_df['filenames']=prediccion_df.apply(lambda row: row.filepath[0:-4].split('/')[-1], axis=1) #Separamos por barraBaja, quitando .JPG

In [ ]:
best_pred = prediccion_df.head(10)
worst_pred = prediccion_df.tail(10).sort_values(by=['abs_error'], ascending=False)

In [ ]:
print(best_pred)

In [ ]:
def generate_saliency(FILEPATH, FILENAME, pred_value, true_value, TYPE):
    saliency_types = [GradientSaliency, IntegratedGradients]

    figsize = 5
    fig, axs = plt.subplots(2, len(saliency_types)+1, figsize=(figsize*(len(saliency_types)+1), figsize*2))
    top_title = "Peso Original: "+str(true_value)+"\nPeso Predicho: "+str(pred_value)
    fig.suptitle(top_title, fontsize=24, y=1.05) #Titulo General TODO: Probar...
    gs = axs[0, 0].get_gridspec()
    # remove the underlying axes
    for ax in axs[0:, 0]:
        ax.remove()
    axbig = fig.add_subplot(gs[0:, 0])

    # Load the image and preprocess it for the saliency maps computation
    data_gen = k_crop_data_sequence(inputs=[FILEPATH],
                                    im_size=conf['model']['image_size'],
                                    mean_RGB=conf['dataset']['mean_RGB'],
                                    std_RGB=conf['dataset']['std_RGB'],
                                    preprocess_mode=conf['model']['preprocess_mode'],
                                    aug_params=None,
                                    crop_mode='random',
                                    crop_number=1)

    img_arr = data_gen.__getitem__(0)
    img_arr = img_arr.squeeze(axis=0)

    # Original image
    image = data_utils.load_image(FILEPATH)#
    image = data_utils.resize_im(image, height=conf['model']['image_size'], width=conf['model']['image_size'])

    axbig.imshow(image)
    axbig.set_title('Original image', fontsize=32, pad=18)
    axbig.set_xticks([])
    axbig.set_yticks([])
    axbig.xaxis.set_visible(False)
    axbig.yaxis.set_visible(False)
    axs = axs.T.flatten()
    # fig.delaxes(axs[1])
    axs = axs[2:]

    # Saliency maps
    # right
    axs[2].set_ylabel('Standard', fontsize=31,rotation=270, labelpad=31)
    axs[3].set_ylabel('Smoothed', fontsize=31,rotation=270, labelpad=31)
    # left
    # axs[0].set_ylabel('Standard', fontsize=12)
    # axs[1].set_ylabel('Smoothed', fontsize=12)
    for i, f in enumerate(saliency_types):
        print('[{}/{}] {}'.format(i+1, len(saliency_types), f.__name__))
        saliency_func = f(model)

        # Normal map
        mask = saliency_func.get_mask(img_arr)
        mask = np.sum(np.abs(mask), axis=2)
        axs[i*2].imshow(mask, cmap=plt.cm.gray, vmin=np.amin(mask), vmax=np.percentile(mask, 98))
        axs[i*2].set_title(saliency_func.__class__.__name__, fontsize=32, pad=18)

        # Smoothgrad map
        mask = saliency_func.get_smoothed_mask(img_arr)
        mask = np.sum(np.abs(mask), axis=2)
        axs[i*2+1].imshow(mask, cmap=plt.cm.gray, vmin=np.amin(mask), vmax=np.percentile(mask, 98))

    # remove the x and y ticks
    for ax in axs:
        ax.set_xticks([])
        ax.set_yticks([])
        ax.yaxis.set_label_position("right")

    fig.tight_layout(pad=1.0)
    result_path = '/srv/image-results/saliencyMaps/'+TYPE+'/'+FILENAME
    plt.savefig(result_path,bbox_inches='tight')
    # Print predicted labels
    #pred_value = predict(model, FILEPATH, conf)
    #print('Predicted value:')
    #print('{}'.format(pred_value[0]))

In [ ]:
if not os.path.exists("/srv/image-results"):
    os.mkdir("/srv/image-results")
if not os.path.exists("/srv/image-results/saliencyMaps"):
    os.mkdir("/srv/image-results/saliencyMaps")
if not os.path.exists("/srv/image-results/saliencyMaps/best"):
    os.mkdir("/srv/image-results/saliencyMaps/best")
if not os.path.exists("/srv/image-results/saliencyMaps/worst"):
    os.mkdir("/srv/image-results/saliencyMaps/worst")

## Best predictions

In [ ]:
for filepath,filename, pred_value, true_value in zip(best_pred['filepath'],best_pred['filenames'],best_pred['pred_value'],best_pred['true_value']) :
    #print(filepath,filename, pred_value, true_value)
    # Generate_saliency
    pred_value = float("{:.2f}".format(pred_value))
    true_value = float("{:.2f}".format(true_value))
    generate_saliency(filepath, filename, pred_value, true_value, "best")

## Worst predictions

In [ ]:
print(worst_pred)

In [ ]:
#Worst predictions
for filepath,filename, pred_value, true_value in zip(worst_pred['filepath'],worst_pred['filenames'],worst_pred['pred_value'],worst_pred['true_value']) :
    #print(filepath,filename, pred_value, true_value)
    # Generate_saliency
    pred_value = float("{:.2f}".format(pred_value))
    true_value = float("{:.2f}".format(true_value))
    generate_saliency(filepath, filename, pred_value, true_value, "worst")